In [25]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import time
import pandas as pd

In [17]:
# Initialize PyMongo to work with MongoDBs
#conn = 'mongodb://localhost:27017'
#client = pymongo.MongoClient(conn)

In [26]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017/mars_app'
client = pymongo.MongoClient(conn)
db = client.mars_app
listings = db.listings.find_one()

In [18]:
# Define database and collection
#db = client.mars
#posts = db.posts

In [35]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [28]:
# URL of page to be scraped
nasa_url = "https://mars.nasa.gov/news"
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
twit_url = "https://twitter.com/marswxreport?lang=en"
facts_url = "https://space-facts.com/mars/"
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [37]:
########
# begin logic for: nasa_url = "https://mars.nasa.gov/news"
#
try:
    browser.visit(nasa_url)
    time.sleep(2)
    soup = BeautifulSoup(browser.html, 'html.parser')
    #response = requests.get(nasa_url)
    time.sleep(2)
    #soup = BeautifulSoup(response.text, 'html.parser')
    #print(soup.prettify())
    txtdiv = soup.find('div', class_="list_text")
    date = txtdiv.find('div', class_="list_date").text
    desc = txtdiv.find('div', class_="article_teaser_body").text
    titlediv = txtdiv.find('div', class_="content_title")
    title = titlediv.text
    href = titlediv.find('a')

    imgdiv = soup.find('div', class_="list_image")
    img = imgdiv.find('img')

    # Print results only if title, price, and link are available
    if ( title and desc):
        print('-------------')
        print('scraping: ' + nasa_url)
        print(date)
        print(title)
        print(desc)
        # Dictionary to be inserted as a MongoDB document
        listings['news_title'] = title
        listings["news_p"] = desc
        listings["news_thumb"] = "https://mars.nasa.gov" + img['src']
        listings["news_link"] = "https://mars.nasa.gov" + href['href']
        listings["date"] = date
    
    db.listings.update({}, listings, upsert=True)
except AttributeError as e:
    print(e)
#
# end logic for: nasa_url = "https://mars.nasa.gov/news"
########

-------------
scraping: https://mars.nasa.gov/news
July 14, 2020
6 Things to Know About NASA's Ingenuity Mars Helicopter
The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.


In [39]:
########
# begin logic for: jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
#
try:
    browser.visit(jpl_url)
    time.sleep(1)
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(1)
    browser.click_link_by_partial_text('more info')
    time.sleep(2)
    soup1 = BeautifulSoup(browser.html, 'html.parser')
    main_img = soup1.find('img', class_="main_image")
    href = main_img['src']
    title = main_img['title']
    print('-----------')
    print('scraping: ' + jpl_url)
    print(title)
    print('https://www.jpl.nasa.gov' + href)
    listings['featured_image_url'] = 'https://www.jpl.nasa.gov'+href
    db.listings.update({}, listings, upsert=True)
except Exception as e:
        print("#### Error: " + e)
#
# end logic for: jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
########

-----------
scraping: https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
Using publicly available data, astronomers have confirmed K2's first exoplanet discovery proving Kepler can still find planets.
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18904_hires.jpg


In [41]:
########
# begin logic for: twit_url = "https://twitter.com/marswxreport?lang=en"
#
browser.visit(twit_url)
time.sleep(4)
soup2 = BeautifulSoup(browser.html, 'html.parser')
firstTime = True
print('-----------------')
print('scraping: ' + twit_url)
for article in soup2.find_all('article'):
    print('-----------------')
    try:
        inSightStr = article.text
        splitstr = inSightStr.split('InSight')[1]
        postinfo = inSightStr.split('InSight')[0]
        print(postinfo)
        print(splitstr)
        
        if ( firstTime ) :
            listings['mars_weather'] = splitstr
            listings['postinfo'] = postinfo
            db.listings.update({}, listings, upsert=True)
            firstTime = False
    except:
        print("Nothing to scrape for this twit")
#
# end logic for: twit_url = "https://twitter.com/marswxreport?lang=en"
########

-----------------
scraping: https://twitter.com/marswxreport?lang=en
-----------------
Mars Weather@MarsWxReport·10h
 sol 579 (2020-07-13) low -88.5ºC (-127.4ºF) high -3.7ºC (25.4ºF)
winds from the WNW at 5.0 m/s (11.2 mph) gusting to 14.8 m/s (33.1 mph)
pressure at 7.80 hPa219
-----------------
Mars Weather@MarsWxReport·Jul 13
 sol 578 (2020-07-12) low -88.3ºC (-126.9ºF) high -5.2ºC (22.7ºF)
winds from the WNW at 5.0 m/s (11.1 mph) gusting to 13.4 m/s (30.1 mph)
pressure at 7.70 hPa621
-----------------
Nothing to scrape for this twit
-----------------
Mars Weather@MarsWxReport·Jul 1
 sol 566 (2020-06-29) low -88.4ºC (-127.1ºF) high -4.2ºC (24.4ºF)
winds from the WNW at 5.1 m/s (11.5 mph) gusting to 16.8 m/s (37.6 mph)
pressure at 7.70 hPa1440
-----------------
Nothing to scrape for this twit
-----------------
Nothing to scrape for this twit
-----------------
Mars Weather@MarsWxReport·Jun 30
 sol 565 (2020-06-28) low -89.6ºC (-129.3ºF) high -5.0ºC (22.9ºF)
winds from the WNW at 4.5 m/

In [42]:
########
# begin logic for: usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
#
hemisphere_image_urls = []
browser.visit(usgs_url)
time.sleep(2)
soup3 = BeautifulSoup(browser.html, 'html.parser')
for item in soup3.find_all('div', class_='item'):
    print('-----------------')
    try:
        img = item.find('img', class_="thumb")
        href = img['src']
        alink = item.find('h3')
        txt = alink.text
        print('-----------')
        print(txt)
        print('https://www.jpl.nasa.gov' + href)
        browser.click_link_by_partial_text(txt)
        time.sleep(2)
        soup3 = BeautifulSoup(browser.html, 'html.parser')
        div = soup3.find('div', class_="downloads")
        full = div.find('a')
        print(full['href'])
        #hemisphere_image_urls.append({'title': txt, 'img_url': full['href']})
        hemisphere_image_urls.append(full['href'])
        browser.back()
    except:
        print("Scraping Complete")
# in case their website is down... which happened on 7/14/20
if ( not hemisphere_image_urls ) : 
    print("### usgs_url unresponsive again!!! ###")
    hemisphere_image_urls.append("http://placehold.it/400x400")
    hemisphere_image_urls.append("http://via.placeholder.com/400")
    hemisphere_image_urls.append("http://placehold.it/400x400")
    hemisphere_image_urls.append("http://via.placeholder.com/400")

listings['hemisphere_image_urls'] = hemisphere_image_urls
db.listings.update({}, listings, upsert=True)
#
# end logic for: usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars""
########

-----------------
-----------
Cerberus Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
-----------------
-----------
Schiaparelli Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
-----------------
-----------
Syrtis Major Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
-----------------
-----------
Valles Marineris Hemisphere Enhanced
https://www.jpl.nasa.gov/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png
https://astropedia.astrogeology.u

{'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}

In [43]:
########
# begin logic for: facts_url = "https://space-facts.com/mars/"
#
tables = pd.read_html(facts_url)
df = tables[0]
df.columns = ['description', 'value']
df

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [44]:
df.set_index('description', inplace=True)

html_table = df.to_html(classes='table table-striped table-hover')
html_table.replace('\n', '')
listings['mars_facts'] = html_table
db.listings.update({}, listings, upsert=True)
df.to_html('table.html', classes='table table-striped table-hover')
#
# end logic for: facts_url = "https://space-facts.com/mars/"
########

In [45]:
# Display items in MongoDB collection

#listings = db.posts.find()
listings = db.listings.find()
for listing in listings:
        print(listing)

print(db.listings.count())
#bills_post = db.listings.find_one({'_id': 1})
#db.posts.drop()

{'_id': ObjectId('5f0e4984126a002f74a45a88'), 'news_title': "6 Things to Know About NASA's Ingenuity Mars Helicopter", 'news_p': 'The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.', 'news_thumb': 'https://mars.nasa.gov/system/news_items/list_view_images/8712_PIA23720-320x240.jpg', 'news_link': 'https://mars.nasa.gov/news/8712/6-things-to-know-about-nasas-ingenuity-mars-helicopter/', 'date': 'July 14, 2020', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18904_hires.jpg', 'mars_weather': ' sol 579 (2020-07-13) low -88.5ºC (-127.4ºF) high -3.7ºC (25.4ºF)\nwinds from the WNW at 5.0 m/s (11.2 mph) gusting to 14.8 m/s (33.1 mph)\npressure at 7.80 hPa219', 'postinfo': 'Mars Weather@MarsWxReport·10h', 'mars_facts': '<table border="1" class="dataframe table table-striped table-hover">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n  

In [46]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017/mars_app'
client = pymongo.MongoClient(conn)
db = client.mars_app

listings = db.listings.find_one()
if ( not listings ) : listings = {}
if ( 'news_title' in listings.keys() ) :
    print('news_title = ' + listings['news_title'])
if ( 'news_p' in listings.keys() ) :
    print('news_p = ' + listings['news_p'])
if ( 'featured_image_url' in listings.keys() ) :
    print('featured_image_url = ' + listings['featured_image_url'])
if ( 'mars_weather' in listings.keys() ) :
    print('mars_weather = ' + listings['mars_weather'])
if ( 'hemisphere_image_urls' in listings.keys() ) :
    print(listings['hemisphere_image_urls'])
if ( 'mars_facts' in listings.keys() ) :
    print('mars_facts = ' + listings['mars_facts'])
print(listings.keys())

news_title = 6 Things to Know About NASA's Ingenuity Mars Helicopter
news_p = The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.
featured_image_url = https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18904_hires.jpg
mars_weather =  sol 579 (2020-07-13) low -88.5ºC (-127.4ºF) high -3.7ºC (25.4ºF)
winds from the WNW at 5.0 m/s (11.2 mph) gusting to 14.8 m/s (33.1 mph)
pressure at 7.80 hPa219
['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']
mars_facts = <table border="1" class="dataframe table table-striped table-hover">
  <thead>
    <tr style="text